In [7]:
import pandas as pd
from word2number import w2n
from nltk.tokenize import word_tokenize

pd.options.display.max_rows = 4000
pd.set_option('display.max_colwidth', None)

def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

def marks_YoE(df):
    
    counter = 0
    
    years_to_colum = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']
    
#     tag_filters = ['sql','python','airflow','etl','snowflake','aws','azure','gcp','bigquery','spark',
#                         'hadoop','hive','lambda','dbt', 'google','amazon','microsoft','bi','tableau',
#                    'power','looker', 'excel','javascript','react','vue']
    
    years = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
             'one','two','three','four','five','six','seven','eight','nine','ten','eleven','twelve',
            'thirteen','fourteen','fifteen']
        
    for year in years_to_colum:
        df[year]=0
        
#     for tag in tag_filters:
#         df[tag]=0
    
    for i in range(df.shape[0]):
        paraSplitted = df.loc[i,'description'].replace(","," ").replace('+'," ").replace('/'," ").replace("'"," ").replace("-"," ").replace('('," ").replace(')'," ").lower().split()
        
        paraSplitted = [word.replace(","," ").replace('+'," ").replace('/'," ").replace("'"," ").replace('-','') for word in paraSplitted]
        
        #         # Marking tags
        #         for word in paraSplitted:
        #             for tag in tag_filters:
        #                 if tag == word:
        #                     df.loc[i,tag]=1
        
        
        # Marking years
        expIndices = [i for i in range(len(paraSplitted)) if 'experience' == paraSplitted[i]]
        #print(expIndices)
        counter = counter + len(expIndices)
        blank_exp = 0
        for instance_of_exp in expIndices:
            count = 0
            for year in years:
                #print(year)
                
                # ex: " 2-5 of experience..."; no line will mention more than 2 numbers 
                #if count == 2:
                #    break
                
                # Marks years 1 -15
                if year in paraSplitted[instance_of_exp-10:instance_of_exp+10]:
                    #print(paraSplitted[instance_of_exp-10:instance_of_exp+10])
                    #print('year: '+year + ', index: '+ str(instance_of_exp))
                    # Marks single digit years, 1 - 9
                    if len(year) == 1 or len(year)==2:
                        df.loc[i,str(year)]=1
                        count += 1
                        
                    # Marks years 10-15
                    else:
                        df.loc[i,str(w2n.word_to_num(year))]=1
                        count += 1

                        
        # Marks jobs with no specific YoE req.
        if 1 not in [df.loc[i,str(j)] for j in range(1,16)]:
            df.loc[i,'0']=1
    return df

def remove_titles(df3):
    rejectTitles = ['manager','summer','intern/co-op','co-op',
                    'coop', 'student', 'intership', 'vice president',
                    'vp','intern','senior','sr','sr.','director',
                    'principal','architect', 'lead']       

    #cleanTitlesInd = pd.DataFrame()
    df3['remove_titles']=0
    for i in range(df3.shape[0]):
        title_tokens = word_tokenize(df3.title[i].replace('-',''))
        

        title_tokens_clean = [word.lower() for word in title_tokens]
        #print(title_tokens_clean)
        if (any(word in title_tokens_clean for word in rejectTitles)):

            df3.loc[i,'remove_titles']=1
            #cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
            #print(df3.loc[i,'title'])
    return df3


def oneTo4(filepath):
    df = remove_titles(marks_YoE(pd.read_json(filepath)))
    df = df.drop_duplicates('job_id').drop_duplicates(subset=['title','company']).reset_index(drop=True)
    return df.loc[ (df["5"]==0)&
                   (df["6"]==0 )&
                   (df["7"]==0 )&
                   (df["8"]==0 )&
                   (df["9"]==0 )&
                   (df["10"]==0) &
                   (df["11"]==0) &
                   (df["12"]==0) &
                   (df["13"]==0) &
                   (df["14"]==0) &
                   (df["15"]==0) 
                   & (df['remove_titles']=0)
                  & (df["employmentType"] != 'contract')
                  #(df["cleanIndustry"]!='it services and it consulting')
                   
                  ,["title","company","noApplicants","postedTimeAgo","job_link", "5"]].sort_values(by='noApplicants').reset_index().style.format({'job_link': make_clickable})

def oneTo3(filepath):
    df = remove_titles(marks_YoE(pd.read_json(filepath)))
    df = df.drop_duplicates('job_id').drop_duplicates(subset=['title','company']).reset_index(drop=True)
    df['postedTimeAgo'] = df['postedTimeAgo'].astype(int)
    return df.loc[
                  (df["4"]==0)&
                    (df["5"]==0)&
                   (df["6"]==0 )&
                   (df["7"]==0 )&
                   (df["8"]==0 )&
                   (df["9"]==0 )&
                   (df["10"]==0) &
                   (df["11"]==0) &
                   (df["12"]==0) &
                   (df["13"]==0) &
                   (df["14"]==0) &
                   (df["15"]==0) 
                    & (df['remove_titles']=0)
                  #(df["cleanIndustry"]!='it services and it consulting')
                   & (df['postedTimeAgo'] <= 12)
                  ,["title",'remove_titles',"company","noApplicants","postedTimeAgo","job_link", "5"]].sort_values(by='noApplicants').reset_index().style.format({'job_link': make_clickable})

def oneTo2(filepath):
    df = remove_titles(marks_YoE(pd.read_json(filepath)))
    df = df.drop_duplicates('job_id').drop_duplicates(subset=['title','company']).reset_index(drop=True)
    return df.loc[(df["3"]==0)&
                  (df["4"]==0)&
                    (df["5"]==0)&
                   (df["6"]==0 )&
                   (df["7"]==0 )&
                   (df["8"]==0 )&
                   (df["9"]==0 )&
                   (df["10"]==0) &
                   (df["11"]==0) &
                   (df["12"]==0) &
                   (df["13"]==0) &
                   (df["14"]==0) &
                   (df["15"]==0)
                   & (df['remove_titles']=0)
                  #(df["cleanIndustry"]!='it services and it consulting')
                   
                  ,["title","company","noApplicants","postedTimeAgo","job_link", "5"]].sort_values(by='noApplicants').reset_index().style.format({'job_link': make_clickable})

def one(filepath):
    df = remove_titles(marks_YoE(pd.read_json(filepath)))
    df = df.drop_duplicates('job_id').drop_duplicates(subset=['title','company']).reset_index(drop=True)
    return df.loc[(df["2"]==0)&(df["3"]==0)&
                  (df["4"]==0)&
                    (df["5"]==0)&
                   (df["6"]==0 )&
                   (df["7"]==0 )&
                   (df["8"]==0 )&
                   (df["9"]==0 )&
                   (df["10"]==0) &
                   (df["11"]==0) &
                   (df["12"]==0) &
                   (df["13"]==0) &
                   (df["14"]==0) &
                   (df["15"]==0) 
                   & (df['remove_titles']=0)
                  #(df["cleanIndustry"]!='it services and it consulting')
                   
                  ,["title","company","noApplicants","postedTimeAgo","job_link", "5"]].sort_values(by='noApplicants').reset_index().style.format({'job_link': make_clickable})





In [37]:
#df = marks_YoE(pd.read_json('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/etlJan27.json')[95:97])
df = pd.read_json('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/2ndFeb_0353AM_analystAND_sqlORpython_.json').reset_index()
df = remove_titles(marks_YoE(df))
df = df.drop_duplicates('job_id').drop_duplicates(subset=['title','company']).reset_index(drop=True)
df.loc[
                  (df["4"]==0)&
                    (df["5"]==0)&
                   (df["6"]==0 )&
                   (df["7"]==0 )&
                   (df["8"]==0 )&
                   (df["9"]==0 )&
                   (df["10"]==0) &
                   (df["11"]==0) &
                   (df["12"]==0) &
                   (df["13"]==0) &
                   (df["14"]==0) &
                   (df["15"]==0) 
                
    & (df['company']=='ipex group of companies')
                  #(df["cleanIndustry"]!='it services and it consulting')
                   
                  ,["title","company","6","noApplicants","postedTimeAgo","job_link", "5"]].sort_values(by='noApplicants').reset_index().style.format({'job_link': make_clickable})

SyntaxError: invalid syntax (1961188528.py, line 22)

In [17]:
z('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/dataAnalystTorontoJan20.json')

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

,index,title,company,noApplicants,postedTimeAgo,job_link
0,0,data analyst - canadian tire corporation,canadian tire corporation,0,48,https://ca.linkedin.com/jobs/view/data-analyst-canadian-tire-corporation-at-canadian-tire-corporation-3442930673?refId=Cq4ppbk4N3yhdrswgKF67Q%3D%3D&trackingId=6ISTFoLnY6s2BCbJqkaldw%3D%3D&position=22&pageNum=0&trk=public_jobs_jserp-result_search-card
1,51,"supervisor, clerical and registration",humber river hospital,0,144,https://ca.linkedin.com/jobs/view/supervisor-clerical-and-registration-at-humber-river-hospital-3433278633?refId=4hba8U2kGo8A7odPH9%2BLWg%3D%3D&trackingId=yK61jxk7Cf2XcEGkLWT5ag%3D%3D&position=7&pageNum=2&trk=public_jobs_jserp-result_search-card
2,48,virtual administrative assistant - full time (remote),clevrecruity,0,1,https://ca.linkedin.com/jobs/view/virtual-administrative-assistant-full-time-remote-at-clevrecruity-3446520766?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=cL8DRt6RIRRxAVdshFtm8w%3D%3D&position=17&pageNum=1&trk=public_jobs_jserp-result_search-card
3,32,developer - specialty / niche,the ian martin group,0,144,https://ca.linkedin.com/jobs/view/developer-specialty-niche-at-the-ian-martin-group-3436162530?refId=4hba8U2kGo8A7odPH9%2BLWg%3D%3D&trackingId=EsdD34mmq%2Fgh3l%2BCqQBGnw%3D%3D&position=6&pageNum=2&trk=public_jobs_jserp-result_search-card
4,30,bilingual business data analyst,robertson & company ltd.,0,72,https://ca.linkedin.com/jobs/view/bilingual-business-data-analyst-at-robertson-company-ltd-3438916874?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=0cICKI0GOl9ldUZStQEj0w%3D%3D&position=16&pageNum=1&trk=public_jobs_jserp-result_search-card
5,28,reporting developer/analyst,motion recruitment,0,24,https://ca.linkedin.com/jobs/view/reporting-developer-analyst-at-motion-recruitment-3444174789?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=KsMlN%2BDGvDCyN9RrwMCrvg%3D%3D&position=13&pageNum=1&trk=public_jobs_jserp-result_search-card
6,26,risk analyst - market data,aston carter,0,6,https://ca.linkedin.com/jobs/view/risk-analyst-market-data-at-aston-carter-3446254584?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=sB1vna8%2FdjUbM7QjvQcl3g%3D%3D&position=20&pageNum=1&trk=public_jobs_jserp-result_search-card
7,23,digital email marketing data analyst (revenue) - hybrid,the globe and mail,0,48,https://ca.linkedin.com/jobs/view/digital-email-marketing-data-analyst-revenue-hybrid-at-the-globe-and-mail-3437845917?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=3ysWvlCPXSYu5zpnd4345Q%3D%3D&position=10&pageNum=1&trk=public_jobs_jserp-result_search-card
8,21,part-time | online data analyst - french (ca),telus international ai data solutions,0,48,https://ca.linkedin.com/jobs/view/part-time-online-data-analyst-french-ca-at-telus-international-ai-data-solutions-3442071300?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=XBipha3I4pgwCef2Mx%2FJBA%3D%3D&position=19&pageNum=1&trk=public_jobs_jserp-result_search-card
9,1,data analyst – coder - decision support - regular full-time (230136),sunnybrook,25,24,https://ca.linkedin.com/jobs/view/data-analyst-%E2%80%93-coder-decision-support-regular-full-time-230136-at-sunnybrook-3444370305?refId=BCNaKAnzih%2BZwY9Wu6USCw%3D%3D&trackingId=cvhprfXaCxdAVIg9XMTUpQ%3D%3D&position=4&pageNum=1&trk=public_jobs_jserp-result_search-card


In [18]:
z('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/dataAnalystTorontoJan20_incognito.json')

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/4041927152.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

,index,title,company,noApplicants,postedTimeAgo,job_link
0,42,reporting developer/analyst,motion recruitment,0,24,https://ca.linkedin.com/jobs/view/reporting-developer-analyst-at-motion-recruitment-3444174789?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=osL5ONHGfk%2F2zrYeS%2Bzmdw%3D%3D&position=13&pageNum=1&trk=public_jobs_jserp-result_search-card
1,50,data analyst - canadian tire corporation,canadian tire corporation,0,48,https://ca.linkedin.com/jobs/view/data-analyst-canadian-tire-corporation-at-canadian-tire-corporation-3442930673?refId=q49QX%2BNNbVtEr9NvJNQPSg%3D%3D&trackingId=047UjYm2FaTyJcoDcn4Wgw%3D%3D&position=18&pageNum=0&trk=public_jobs_jserp-result_search-card
2,43,"supervisor, clerical and registration",humber river hospital,0,144,https://ca.linkedin.com/jobs/view/supervisor-clerical-and-registration-at-humber-river-hospital-3433278633?refId=bQu%2F5WAs7nJkXhZsp%2FVwqg%3D%3D&trackingId=qvRscloTiWA0ciBPQAvP2g%3D%3D&position=7&pageNum=2&trk=public_jobs_jserp-result_search-card
3,18,digital email marketing data analyst (revenue) - hybrid,the globe and mail,0,48,https://ca.linkedin.com/jobs/view/digital-email-marketing-data-analyst-revenue-hybrid-at-the-globe-and-mail-3437845917?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=%2B2qzGIz13KOlgd5k8etwmA%3D%3D&position=10&pageNum=1&trk=public_jobs_jserp-result_search-card
4,6,risk analyst - market data,aston carter,0,6,https://ca.linkedin.com/jobs/view/risk-analyst-market-data-at-aston-carter-3446254584?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=G%2BQ7NHcc0ANi5yNVz7IcQg%3D%3D&position=20&pageNum=1&trk=public_jobs_jserp-result_search-card
5,33,part-time | online data analyst - french (ca),telus international ai data solutions,0,48,https://ca.linkedin.com/jobs/view/part-time-online-data-analyst-french-ca-at-telus-international-ai-data-solutions-3442071300?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=X99zhiK1uleaVjrKHum09w%3D%3D&position=19&pageNum=1&trk=public_jobs_jserp-result_search-card
6,15,virtual administrative assistant - full time (remote),clevrecruity,0,1,https://ca.linkedin.com/jobs/view/virtual-administrative-assistant-full-time-remote-at-clevrecruity-3446520766?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=s8p4kQb3WPPrznsvXFkZIw%3D%3D&position=17&pageNum=1&trk=public_jobs_jserp-result_search-card
7,29,developer - specialty / niche,the ian martin group,0,144,https://ca.linkedin.com/jobs/view/developer-specialty-niche-at-the-ian-martin-group-3436162530?refId=bQu%2F5WAs7nJkXhZsp%2FVwqg%3D%3D&trackingId=fixkUtXA0mU0LQZzhG3Tew%3D%3D&position=6&pageNum=2&trk=public_jobs_jserp-result_search-card
8,35,bilingual business data analyst,robertson & company ltd.,0,72,https://ca.linkedin.com/jobs/view/bilingual-business-data-analyst-at-robertson-company-ltd-3438916874?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=xVDuiryAZNUwK0A9D%2FF9KQ%3D%3D&position=16&pageNum=1&trk=public_jobs_jserp-result_search-card
9,13,data analyst – coder - decision support - regular full-time (230136),sunnybrook,25,24,https://ca.linkedin.com/jobs/view/data-analyst-%E2%80%93-coder-decision-support-regular-full-time-230136-at-sunnybrook-3444370305?refId=K3RNKLazL78bnTHVGLabCg%3D%3D&trackingId=kIrzXCmrtClwDchAfkfxdw%3D%3D&position=4&pageNum=1&trk=public_jobs_jserp-result_search-card


In [13]:
df1 = remove_titles(marks_YoE(pd.read_json('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/Jan22_10_33_PM_AnalystANDdata.json')))
df2 = remove_titles(marks_YoE(pd.read_json('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/Jan22_10_33_PM_AnalystANDexcel.json')))
df = pd.concat([df1, df2]).drop_duplicates('job_id').reset_index(drop=True)
df.loc[(df["0"]==1) |
      (df["1"]==1) |
      (df["2"]==1) |
      (df["3"]==1) | 
      (df["4"]==1)|
       (df["cleanIndustry"]=='it services and it consulting')
       ,
      ["title","company","noApplicants","postedTimeAgo",
       "job_link"]].sort_values(by='noApplicants').reset_index().style.format({'job_link': make_clickable})


/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

...

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_2440/901683900.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

,index,title,company,noApplicants,postedTimeAgo,job_link
0,221,canadian hr/payroll business analyst,sability,0,120,https://ca.linkedin.com/jobs/view/canadian-hr-payroll-business-analyst-at-sability-3441661819?refId=y5zNUu49JOFzBsbumOpvnQ%3D%3D&trackingId=qPgNmolfN3fXT74PBy7%2FgA%3D%3D&position=16&pageNum=9&trk=public_jobs_jserp-result_search-card
1,490,data migration consultant,atlantis it group,0,48,https://ca.linkedin.com/jobs/view/data-migration-consultant-at-atlantis-it-group-3444242206?refId=cb%2Bp6gKtq49zCK%2FVjV0EsQ%3D%3D&trackingId=mI5V5iSbhoZZcD6zpH%2B%2BJw%3D%3D&position=22&pageNum=20&trk=public_jobs_jserp-result_search-card
2,351,"business analyst (bangkok based, relocation provided)",agoda,0,48,https://ca.linkedin.com/jobs/view/business-analyst-bangkok-based-relocation-provided-at-agoda-3443483632?refId=t45VW8Tkilkx1%2FnSpvLcCw%3D%3D&trackingId=Lrp0RRy25uWjnc4gldCO5Q%3D%3D&position=1&pageNum=23&trk=public_jobs_jserp-result_search-card
3,176,"analyst, dynamic currency conversion - bilingual english/spanish",cibc,0,24,https://ca.linkedin.com/jobs/view/analyst-dynamic-currency-conversion-bilingual-english-spanish-at-cibc-3446170139?refId=38eCQoNHozwfjPFGhXiWVA%3D%3D&trackingId=vPdYb1SfGhMrmiDaKTF6Qw%3D%3D&position=19&pageNum=16&trk=public_jobs_jserp-result_search-card
4,349,it product owner bilingual(spanish and english),delpath,0,48,https://ca.linkedin.com/jobs/view/it-product-owner-bilingual-spanish-and-english-at-delpath-3447239140?refId=Vwmv1datHkuXBZr3Aa1NAA%3D%3D&trackingId=4nOCh9rAhlMzokaeppYQeg%3D%3D&position=22&pageNum=31&trk=public_jobs_jserp-result_search-card
5,348,brand advocate supervisor,apex systems,0,48,https://ca.linkedin.com/jobs/view/brand-advocate-supervisor-at-apex-systems-3443340981?refId=Vwmv1datHkuXBZr3Aa1NAA%3D%3D&trackingId=RGvQQ2HKfOjsT%2Bl49SngQg%3D%3D&position=17&pageNum=31&trk=public_jobs_jserp-result_search-card
6,346,"finance systems analyst (bangkok based, relocation provided)",agoda,0,48,https://ca.linkedin.com/jobs/view/finance-systems-analyst-bangkok-based-relocation-provided-at-agoda-3443486211?refId=xnHzAE4qkwwN3P02C8QKeA%3D%3D&trackingId=i144CN9F4oqVrxJBSx4yVw%3D%3D&position=2&pageNum=22&trk=public_jobs_jserp-result_search-card
7,343,technical analyst - procurement with top 5 bank,aston carter,0,5,https://ca.linkedin.com/jobs/view/technical-analyst-procurement-with-top-5-bank-at-aston-carter-3449363486?refId=xnHzAE4qkwwN3P02C8QKeA%3D%3D&trackingId=Tja4hrHUYFx%2Bbpwp1Vwv8Q%3D%3D&position=5&pageNum=22&trk=public_jobs_jserp-result_search-card
8,342,associate application support specialist,s&p global,0,24,https://ca.linkedin.com/jobs/view/associate-application-support-specialist-at-s-p-global-3445751295?refId=xnHzAE4qkwwN3P02C8QKeA%3D%3D&trackingId=htfKF02RSSI%2B6jXuOEAEkg%3D%3D&position=14&pageNum=22&trk=public_jobs_jserp-result_search-card
9,186,associate quality engineer- en,td,0,120,https://ca.linkedin.com/jobs/view/associate-quality-engineer-en-at-td-3442717158?refId=8hcXOr%2BCeHQWDxLTDJQ%2FZQ%3D%3D&trackingId=PxioMjYqRNKBHJ76G45mig%3D%3D&position=15&pageNum=28&trk=public_jobs_jserp-result_search-card


In [32]:
z('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/Jan23_2_34_AM_sqlANDexcel.json')

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

,index,title,company,noApplicants,postedTimeAgo,job_link
0,8,analyste informatique,st-amour,0,14,https://ca.linkedin.com/jobs/view/analyste-informatique-at-st-amour-3450402901?refId=88LZ1BySomsKNy5cChROAg%3D%3D&trackingId=E6ZvxktXKI0o3ilW%2F8pE5Q%3D%3D&position=19&pageNum=0&trk=public_jobs_jserp-result_search-card
1,10,technical software analyst,touchtunes,0,10,https://ca.linkedin.com/jobs/view/technical-software-analyst-at-touchtunes-3447085611?refId=88LZ1BySomsKNy5cChROAg%3D%3D&trackingId=aiOeN3ThvzgjaW%2FtPVi9nA%3D%3D&position=15&pageNum=0&trk=public_jobs_jserp-result_search-card
2,23,product owner - 4/ba/bsa (spanish),raise,0,10,https://ca.linkedin.com/jobs/view/product-owner-4-ba-bsa-spanish-at-raise-3449297239?refId=84oLizezFFgjWXz4rfEPdA%3D%3D&trackingId=7cDCV808HISkE2IHmxDuQw%3D%3D&position=11&pageNum=1&trk=public_jobs_jserp-result_search-card
3,35,data center associate,pwc canada,0,4,https://ca.linkedin.com/jobs/view/data-center-associate-at-pwc-canada-3451591312?refId=88LZ1BySomsKNy5cChROAg%3D%3D&trackingId=PF82DwJM6RpwikKahMnH%2FA%3D%3D&position=11&pageNum=0&trk=public_jobs_jserp-result_search-card
4,39,human resources information system specialist,reelcruit,0,8,https://ca.linkedin.com/jobs/view/human-resources-information-system-specialist-at-reelcruit-3451480617?refId=88LZ1BySomsKNy5cChROAg%3D%3D&trackingId=TKyvRGk2Im6Y7UV9F23K3w%3D%3D&position=13&pageNum=0&trk=public_jobs_jserp-result_search-card
5,44,développeur fonctionnel (oipa),equisoft,0,21,https://ca.linkedin.com/jobs/view/d%C3%A9veloppeur-fonctionnel-oipa-at-equisoft-3437963510?refId=YjROLn%2FFeUjzx2w1x25C%2Bg%3D%3D&trackingId=%2Fi9EjfMLDJaO1GsMJJZzwg%3D%3D&position=12&pageNum=2&trk=public_jobs_jserp-result_search-card
6,46,site reliability engineer,intrado,0,14,https://ca.linkedin.com/jobs/view/site-reliability-engineer-at-intrado-3429643428?refId=YjROLn%2FFeUjzx2w1x25C%2Bg%3D%3D&trackingId=KAWHJ8yXrEcEdGgdol5HMQ%3D%3D&position=14&pageNum=2&trk=public_jobs_jserp-result_search-card
7,61,ppq coordinator product safety,brp,0,21,https://ca.linkedin.com/jobs/view/ppq-coordinator-product-safety-at-brp-3449772352?refId=YjROLn%2FFeUjzx2w1x25C%2Bg%3D%3D&trackingId=uBvyck8z2RFYVHVX5CDrEQ%3D%3D&position=19&pageNum=2&trk=public_jobs_jserp-result_search-card
8,66,"analyst, assortment & space planning",loblaw companies limited,0,13,https://ca.linkedin.com/jobs/view/analyst-assortment-space-planning-at-loblaw-companies-limited-3449265316?refId=84oLizezFFgjWXz4rfEPdA%3D%3D&trackingId=uBdW6m1T4m20fAfTCPv3FQ%3D%3D&position=21&pageNum=1&trk=public_jobs_jserp-result_search-card
9,56,site reliability engineer,intrado,26,9,https://ca.linkedin.com/jobs/view/site-reliability-engineer-at-intrado-3429638729?refId=YjROLn%2FFeUjzx2w1x25C%2Bg%3D%3D&trackingId=JuOV%2Fq5ZtOpgpi1rlJ9g0w%3D%3D&position=1&pageNum=2&trk=public_jobs_jserp-result_search-card


In [33]:
z('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/Jan24_8-15-AM_sqlSoftwareIndustry.json')

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleanTitlesInd  = cleanTitlesInd.append(df3.loc[i,:])
/var/folders/k_/h4ll7zh10pb0p9z2t0kh6cm80000gn/T/ipykernel_4232/3107155906.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

,index,title,company,noApplicants,postedTimeAgo,job_link
0,2,delivery consultant - bilingual (french-english),dynatrace,0,2,https://ca.linkedin.com/jobs/view/delivery-consultant-bilingual-french-english-at-dynatrace-3389911487?refId=1ifuYVNFFcZNRDvNu7Ie1w%3D%3D&trackingId=PFPUimkr28N55ZIe50Sirg%3D%3D&position=16&pageNum=5&trk=public_jobs_jserp-result_search-card
1,93,analyste d'affaires ti principal_fr,intact,0,13,https://ca.linkedin.com/jobs/view/analyste-d-affaires-ti-principal-fr-at-intact-3451504774?refId=hE4xRcNtQ2rWvTQQ6T%2Bw8A%3D%3D&trackingId=r%2FLXaqDlCe1CGSgDkpZT0Q%3D%3D&position=10&pageNum=9&trk=public_jobs_jserp-result_search-card
2,95,"gestionnaire de projets, automatisation",centris technologies,0,18,https://ca.linkedin.com/jobs/view/gestionnaire-de-projets-automatisation-at-centris-technologies-3447055439?refId=r3ExTGhBkl9rwiumDdkW7g%3D%3D&trackingId=NRvmSV4Qd6sI0OuCsbM5KQ%3D%3D&position=24&pageNum=1&trk=public_jobs_jserp-result_search-card
3,101,javascript developer,hubelia,0,0,https://ca.linkedin.com/jobs/view/javascript-developer-at-hubelia-3452069332?refId=r3ExTGhBkl9rwiumDdkW7g%3D%3D&trackingId=2sCWpKpSWbfhfpVrcas2lg%3D%3D&position=23&pageNum=1&trk=public_jobs_jserp-result_search-card
4,111,field application engineer,graymatter robotics,0,16,https://ca.linkedin.com/jobs/view/field-application-engineer-at-graymatter-robotics-3443301139?refId=r3ExTGhBkl9rwiumDdkW7g%3D%3D&trackingId=P2MU1EgC4Kv5%2Bp6%2FwmYjNw%3D%3D&position=17&pageNum=1&trk=public_jobs_jserp-result_search-card
5,369,full stack ruby on rails developer,scout talent north america,0,17,https://ca.linkedin.com/jobs/view/full-stack-ruby-on-rails-developer-at-scout-talent-north-america-3447067305?refId=qUBzExidWRgxoXvqVCb2fw%3D%3D&trackingId=fVgMIW8casf6%2FJCVoQE5kw%3D%3D&position=25&pageNum=11&trk=public_jobs_jserp-result_search-card
6,165,spécialiste de validation / validation specialist,ultra forensic technology,0,20,https://ca.linkedin.com/jobs/view/sp%C3%A9cialiste-de-validation-validation-specialist-at-ultra-forensic-technology-3447035637?refId=gCf6jp1DedcT8Rzo3CMyWQ%3D%3D&trackingId=mIfp1BtLbe1koEjnkDNMcw%3D%3D&position=9&pageNum=4&trk=public_jobs_jserp-result_search-card
7,214,total compensation analyst,jobber,0,14,https://ca.linkedin.com/jobs/view/total-compensation-analyst-at-jobber-3451421555?refId=DxXYS1QC%2Bp0WMnrtgwMX2w%3D%3D&trackingId=pOcIPHhJaSNP3o0MBV8qbQ%3D%3D&position=25&pageNum=2&trk=public_jobs_jserp-result_search-card
8,222,analyste d'affaires ti principal_fr,intact,0,13,https://ca.linkedin.com/jobs/view/analyste-d-affaires-ti-principal-fr-at-intact-3451504774?refId=MvH7VQRJBO%2FBVKSVIeOklA%3D%3D&trackingId=ef6Cs3svW60yPeckCOCmkw%3D%3D&position=23&pageNum=10&trk=public_jobs_jserp-result_search-card
9,232,"analyste d'affaires, technologies des placements",intact,0,19,https://ca.linkedin.com/jobs/view/analyste-d-affaires-technologies-des-placements-at-intact-3450411073?refId=MvH7VQRJBO%2FBVKSVIeOklA%3D%3D&trackingId=xT0gl4swhW7ZTkzpn8Gqjw%3D%3D&position=19&pageNum=10&trk=public_jobs_jserp-result_search-card


In [8]:
oneTo3('/Users/mvbasxhr/Cool Stuff/LinkedInScraper/linkedin/2ndFeb_1411AM_analystAND_sqlORpython_.json')


,index,title,remove_titles,company,noApplicants,postedTimeAgo,job_link,5
0,65,geomatics analyst ii,0,clickjobs.io,0,2,https://ca.linkedin.com/jobs/view/geomatics-analyst-ii-at-clickjobs-io-3462729357?refId=WvlfAVwP3nBd4PUDclzAnA%3D%3D&trackingId=yzsvkLu6Y%2FgskazyGNwR5Q%3D%3D&position=13&pageNum=6&trk=public_jobs_jserp-result_search-card,0
1,121,data analyst -finance specialist,0,bull it services,0,0,https://ca.linkedin.com/jobs/view/data-analyst-finance-specialist-at-bull-it-services-3459759596?refId=CRQjRT0FcFLRBCKVY5UU5A%3D%3D&trackingId=YqusPg1z1w1%2FAaxCDg0XPQ%3D%3D&position=6&pageNum=0&trk=public_jobs_jserp-result_search-card,0
2,119,manager- banking technology - capital markets,1,ey,0,4,https://ca.linkedin.com/jobs/view/manager-banking-technology-capital-markets-at-ey-3435487814?refId=WvlfAVwP3nBd4PUDclzAnA%3D%3D&trackingId=vRNfiLymee6DprG4aBCWvw%3D%3D&position=20&pageNum=6&trk=public_jobs_jserp-result_search-card,0
3,129,analyste principal au développement – information de gestion / senior management information development analyst,1,air canada,0,0,https://ca.linkedin.com/jobs/view/analyste-principal-au-d%C3%A9veloppement-%E2%80%93-information-de-gestion-senior-management-information-development-analyst-at-air-canada-3463301488?refId=9fonmHHI8TlzPqZwAa4I3g%3D%3D&trackingId=XPmpImsDlOwqvXk0xDJHjA%3D%3D&position=18&pageNum=5&trk=public_jobs_jserp-result_search-card,0
4,110,hr reporting specialist,0,brp,0,8,https://ca.linkedin.com/jobs/view/hr-reporting-specialist-at-brp-3465241288?refId=rjPLoFgz%2FsSr%2FOap3LNOJg%3D%3D&trackingId=8bYtRHJHjlCzHCtmvFdcvw%3D%3D&position=15&pageNum=4&trk=public_jobs_jserp-result_search-card,0
5,105,migration analyst tester,0,fnz group,0,10,https://ca.linkedin.com/jobs/view/migration-analyst-tester-at-fnz-group-3461660233?refId=rjPLoFgz%2FsSr%2FOap3LNOJg%3D%3D&trackingId=YqM2OUqVHLCAyBRVXiTNUA%3D%3D&position=11&pageNum=4&trk=public_jobs_jserp-result_search-card,0
6,103,"business analyst, order management",0,brp,0,8,https://ca.linkedin.com/jobs/view/business-analyst-order-management-at-brp-3465241332?refId=rjPLoFgz%2FsSr%2FOap3LNOJg%3D%3D&trackingId=rLBSHa012row%2Bk3ikQEzXw%3D%3D&position=3&pageNum=4&trk=public_jobs_jserp-result_search-card,0
7,98,sto 21r - gis analyst (bc public service),0,bcjobs,0,2,https://ca.linkedin.com/jobs/view/sto-21r-gis-analyst-bc-public-service-at-bcjobs-3462721571?refId=9fonmHHI8TlzPqZwAa4I3g%3D%3D&trackingId=3JQwa%2BpHdX9I1i0afFswog%3D%3D&position=11&pageNum=5&trk=public_jobs_jserp-result_search-card,0
8,96,software support analyst/programmer,0,s.i. systems,0,0,https://ca.linkedin.com/jobs/view/software-support-analyst-programmer-at-s-i-systems-3459766858?refId=rjPLoFgz%2FsSr%2FOap3LNOJg%3D%3D&trackingId=2EQnsbfyNMb9bquJBRd4pw%3D%3D&position=10&pageNum=4&trk=public_jobs_jserp-result_search-card,0
9,94,the quality assurance analyst,0,harmonic fund services,0,1,https://ca.linkedin.com/jobs/view/the-quality-assurance-analyst-at-harmonic-fund-services-3459750532?refId=9fonmHHI8TlzPqZwAa4I3g%3D%3D&trackingId=ClnWXQcIl8W%2BB6ZM5wgbpQ%3D%3D&position=22&pageNum=5&trk=public_jobs_jserp-result_search-card,0
